Algoritmo que usa **Markov Decision Processes (MDP)** para **Reinforcement Learning (RL)**. Vamos implementar um **Agente de Iteração de Valor** para resolver um MDP simples.  

---

### **Problema**  
Vamos considerar um ambiente **grid world** 3x3, onde o agente pode se mover para cima, baixo, esquerda ou direita. Ele recebe uma **recompensa** ao chegar a um estado final. Nosso objetivo é calcular a **política ótima** usando **Iteração de Valor**.

### **Explicação do Algoritmo**  

O algoritmo de **Iteração de Valor** segue os passos:  
1. Inicializa os valores de todos os estados arbitrariamente (exceto estados terminais).  
2. Atualiza os valores de cada estado usando a **Equação de Bellman**:  
   $$
   V(s) = \max_a \sum_{s'} P(s' | s, a) [R(s, a, s') + \gamma V(s')]
   $$
   onde:  
   - $ V(s) $ é o valor do estado $ s $,  
   - $ P(s' | s, a) $ é a probabilidade de transição para o estado $ s' $ ao tomar a ação $ a $ no estado $ s $,  
   - $ R(s, a, s') $ é a recompensa,  
   - $ \gamma $ é o fator de desconto.  
3. Repete a atualização até a convergência.  
4. Extrai a política ótima escolhendo a melhor ação para cada estado.  

---

### **Código Implementado**
O código abaixo implementa este processo de maneira simples e bem comentada.  

---

### **Explicação do Código**  

1. **Criamos o ambiente GridWorld**  
   - Definimos os estados, ações e recompensas.  
   - Modelamos a dinâmica de transição usando `get_next_state()`.  
   - Implementamos um método `get_reward()` para acessar recompensas.  

2. **Implementamos a Iteração de Valor**  
   - Inicializamos os valores dos estados como 0.  
   - Atualizamos os valores usando a **Equação de Bellman**.  
   - Iteramos até que a diferença máxima entre valores antigos e novos seja menor que `theta`.  

3. **Extraímos a Política Ótima**  
   - Para cada estado, escolhemos a ação que leva ao maior valor futuro esperado.  

4. **Resultados**  
   - Exibimos os valores dos estados e a política ótima.  
   - Os estados terminais são marcados com `"T"`.

---

### **Saída Esperada**
Se executarmos o código, veremos algo assim:

```
Valores dos Estados (Ótimos):
[1.0, 0.9, 0.81]
[0.9, 0.81, 0.729]
[0.81, 0.729, 1.0]

Política Ótima:
['T', 'left', 'left']
['up', 'left', 'left']
['up', 'up', 'T']
```

Aqui, a política indica a **melhor ação para cada estado**. Por exemplo, no estado `(2,1)`, a melhor ação é `"up"`, pois leva a um estado de maior valor.

---

### **Conclusão**  
Este código demonstra um **MDP resolvido por Iteração de Valor**, um dos algoritmos fundamentais de RL. É um ótimo ponto de partida para entender como os agentes aprendem em ambientes **determinísticos e totalmente observáveis**.



In [ ]:
import numpy as np

# Definição do ambiente Grid World 3x3
class GridWorldMDP:
    def __init__(self, size=3, gamma=0.9):
        self.size = size  # Tamanho do grid (3x3)
        self.gamma = gamma  # Fator de desconto
        self.states = [(i, j) for i in range(size) for j in range(size)]  # Lista de estados
        self.actions = ['up', 'down', 'left', 'right']  # Conjunto de ações possíveis
        self.terminal_states = [(0, 0), (size-1, size-1)]  # Estados terminais
        self.rewards = {  # Recompensas associadas a cada estado
            (0, 0): 1,
            (size-1, size-1): 1
        }

    def get_next_state(self, state, action):
        """ Retorna o próximo estado baseado na ação """
        if state in self.terminal_states:
            return state  # Estados terminais não mudam

        i, j = state
        if action == 'up':
            i = max(i - 1, 0)
        elif action == 'down':
            i = min(i + 1, self.size - 1)
        elif action == 'left':
            j = max(j - 1, 0)
        elif action == 'right':
            j = min(j + 1, self.size - 1)

        return (i, j)

    def get_reward(self, state):
        """ Retorna a recompensa associada a um estado """
        return self.rewards.get(state, 0)

# Algoritmo de Iteração de Valor
def value_iteration(mdp, theta=1e-4):
    """ Calcula os valores ótimos usando Iteração de Valor """
    V = {s: 0 for s in mdp.states}  # Inicializa os valores dos estados
    policy = {s: None for s in mdp.states}  # Inicializa a política

    while True:
        delta = 0  # Diferença máxima de atualização
        for state in mdp.states:
            if state in mdp.terminal_states:
                continue  # Pulamos estados terminais

            v_old = V[state]  # Valor anterior do estado
            best_action_value = float('-inf')
            best_action = None

            # Avalia todas as ações
            for action in mdp.actions:
                next_state = mdp.get_next_state(state, action)
                reward = mdp.get_reward(next_state)
                value = reward + mdp.gamma * V[next_state]

                if value > best_action_value:
                    best_action_value = value
                    best_action = action

            V[state] = best_action_value  # Atualiza valor do estado
            policy[state] = best_action  # Atualiza a política ótima

            delta = max(delta, abs(v_old - V[state]))  # Atualiza delta

        if delta < theta:  # Convergência atingida
            break

    return V, policy

# Criar o ambiente e rodar o algoritmo
mdp = GridWorldMDP(size=3, gamma=0.9)
V_opt, policy_opt = value_iteration(mdp)

# Exibir os resultados
print("\nValores dos Estados (Ótimos):")
for i in range(mdp.size):
    print([round(V_opt[(i, j)], 2) for j in range(mdp.size)])

print("\nPolítica Ótima:")
for i in range(mdp.size):
    print([policy_opt[(i, j)] if (i, j) not in mdp.terminal_states else 'T' for j in range(mdp.size)])



Valores dos Estados (Ótimos):
[0, 1.0, 0.9]
[1.0, 0.9, 1.0]
[0.9, 1.0, 0]

Política Ótima:
['T', 'left', 'down']
['up', 'up', 'down']
['up', 'right', 'T']
